# ReAct: Reasoning + Acting

Building on:
- **001-003**: Basic Branch usage and function calling
- **004**: Conversation patterns and context

**This notebook**: Introduction to ReAct - getting AI to think step-by-step and use tools strategically.

In [1]:
from lionagi import Branch, iModel
import json
from IPython.display import Markdown

## What is ReAct?

**ReAct** = **Reasoning** + **Acting**

Instead of just giving an answer, the AI:
1. **Thinks** about the problem (reasoning)
2. **Uses tools** when needed (acting)
3. **Repeats** until the problem is solved

This creates more reliable, step-by-step problem solving.

## Simple Example: Math Problem Solver

Let's start with a basic calculator that shows its reasoning:

In [2]:
# Simple calculator functions
def add(a: float, b: float) -> float:
    """Add two numbers."""
    return a + b


def multiply(a: float, b: float) -> float:
    """Multiply two numbers."""
    return a * b


def divide(a: float, b: float) -> float:
    """Divide two numbers."""
    if b == 0:
        return "Error: Cannot divide by zero"
    return a / b


# Create a ReAct-enabled branch
calculator = Branch(
    system="You solve math problems step by step. Think about what you need to calculate, then use tools.",
    tools=[add, multiply, divide],
    chat_model=iModel(model="openai/gpt-4.1-mini"),
)

print("Calculator ready with ReAct capabilities!")

Calculator ready with ReAct capabilities!


## Using ReAct for Problem Solving

In [3]:
# Problem: Calculate the total cost of 3 items at $15.50 each, plus 8% tax
problem = (
    "I bought 3 items at $15.50 each. What's the total cost including 8% tax?"
)

# Use ReAct to solve step-by-step
solution = await calculator.ReAct(
    instruct={"instruction": problem},
    max_extensions=5,  # Limit the number of reasoning steps
    verbose=True,  # Show the reasoning process
)

print("\n🎯 Final Answer:")
print(solution)

Invoking action multiply with {'a': 3, 'b': 15.5}.
Action multiply invoked, status: completed.


### ReAct Round No.1 Analysis:
```yaml
analysis: First, calculate the total price for 3 items each costing $15.50 by multiplying 3 by 15.50. Then, calculate the 8% tax on the total price. Finally, add the tax to the total price to get the final cost including tax.
planned_actions:
  - action_type: multiply
    description: Calculate the total price for 3 items at $15.50 each.
extension_needed: True
milestone: Calculate total item cost before tax.
action_strategy: sequential
action_required: True
reason:
  title: Step to find total cost before tax
  content: We need the total item cost before applying the 8% tax to find the final total.
  confidence_score: 1.0
action_requests:
  - function: multiply
    arguments:
      a: 3
      b: 15.5
action_responses:
  - function: multiply
    arguments:
      a: 3
      b: 15.5
    output: 46.5
```
---------

Invoking action multiply with {'a': 46.5, 'b': 0.08}.
Action multiply invoked, status: completed.


### ReAct Round No.2 Analysis:
```yaml
analysis: Next, calculate the 8% tax on the total cost before tax, which is $46.5. This will give us the tax amount to be added to get the final total cost.
planned_actions:
  - action_type: multiply
    description: Calculate the 8% tax on the total item cost of $46.5 by multiplying 46.5 by 0.08.
extension_needed: True
milestone: Calculate tax amount on total item cost.
action_strategy: sequential
action_required: True
reason:
  title: Calculate tax amount
  content: We need to calculate 8% tax on the total item cost to find how much tax is added to the purchase.
  confidence_score: 1.0
action_requests:
  - function: multiply
    arguments:
      a: 46.5
      b: 0.08
action_responses:
  - function: multiply
    arguments:
      a: 46.5
      b: 0.08
    output: 3.72
```
---------

Invoking action add with {'a': 46.5, 'b': 3.72}.
Action add invoked, status: completed.


### ReAct Round No.3 Analysis:
```yaml
analysis: Now that we have the tax amount $3.72, we can add this to the initial total $46.5 to get the final cost including the 8% tax.
planned_actions:
  - action_type: add
    description: Add the tax amount $3.72 to the total item cost $46.5 to get the final total cost including tax.
extension_needed: True
milestone: Calculate final total cost including tax.
action_strategy: sequential
action_required: True
reason:
  title: Calculate final total price
  content: Adding the calculated tax amount to the original total item cost gives the final price including tax.
  confidence_score: 1.0
action_requests:
  - function: add
    arguments:
      a: 46.5
      b: 3.72
action_responses:
  - function: add
    arguments:
      a: 46.5
      b: 3.72
    output: 50.22
```
---------

### ReAct Round No.4 Analysis:
```yaml
analysis: The total cost for 3 items at $15.50 each with an 8% tax included is $50.22. This includes the $46.50 total item price plus $3.72 tax.
planned_actions:
extension_needed: False
milestone: Final total cost including tax calculated.
action_strategy: sequential
action_required: False
reason:
  title: Final calculation completed
  content: All necessary calculations have been performed: total item cost, tax, and final total. The final amount is ready to present.
  confidence_score: 1.0
action_requests:
```
---------

### ReAct Final Answer:
The total cost for 3 items at $15.50 each, including an 8% tax, is $50.22.
---------


🎯 Final Answer:
The total cost for 3 items at $15.50 each, including an 8% tax, is $50.22.


## Understanding the ReAct Process

Let's look at what happened in the conversation:

In [4]:
# View the reasoning steps
conversation = calculator.to_df()
print(f"Total conversation steps: {len(conversation)}")
print("\nStep-by-step breakdown:")

for i, row in conversation.iterrows():
    if row["role"] == "user":
        print(f"\n👤 User: {str(row['content'])[:100]}...")
    elif row["role"] == "assistant":
        print(f"🤖 Assistant: {str(row['content'])[:100]}...")
    elif row["role"] == "action":
        print(f"🔧 Tool Used: {str(row['content'])[:100]}...")

Total conversation steps: 17

Step-by-step breakdown:

👤 User: {'context': [], 'instruction': "I bought 3 items at $15.50 each. What's the total cost including 8% ...
🤖 Assistant: {'assistant_response': '```json\n{"planned_actions":[{"action_type":"multiply","description":"Calcul...
🔧 Tool Used: {'action_request': {'function': 'multiply', 'arguments': {'a': 3, 'b': 15.5}}, 'action_response_id':...
🔧 Tool Used: {'action_request_id': 'ba13cecd-f7ca-4e7d-8a57-3da9b182e183', 'action_response': {'function': 'multi...

👤 User: {'context': [{'action_request_id': 'ba13cecd-f7ca-4e7d-8a57-3da9b182e183', 'action_response': {'func...
🤖 Assistant: {'assistant_response': '```json\n{"planned_actions":[{"action_type":"multiply","description":"Calcul...
🔧 Tool Used: {'action_request': {'function': 'multiply', 'arguments': {'a': 46.5, 'b': 0.08}}, 'action_response_i...
🔧 Tool Used: {'action_request_id': 'f1ee8c36-7267-4af9-9620-3c0732d4f039', 'action_response': {'function': 'multi...

👤 User: {'context

## Practical Example: Research Assistant

Let's build a simple research assistant that can look up information:

In [5]:
# Simple "database" functions
company_data = {
    "apple": {"founded": 1976, "employees": 164000, "revenue": "$394.3B"},
    "google": {"founded": 1998, "employees": 182000, "revenue": "$307.4B"},
    "microsoft": {"founded": 1975, "employees": 238000, "revenue": "$211.9B"},
}


def lookup_company(name: str) -> str:
    """Look up information about a company."""
    name = name.lower()
    if name in company_data:
        data = company_data[name]
        return f"{name.title()}: Founded {data['founded']}, {data['employees']} employees, {data['revenue']} revenue"
    return f"No data found for {name}"


def calculate_age(founded_year: int) -> int:
    """Calculate how old a company is."""
    from datetime import datetime

    current_year = datetime.now().year
    return current_year - founded_year


# Research assistant with ReAct
researcher = Branch(
    system="You are a research assistant. Look up information and analyze it step by step.",
    tools=[lookup_company, calculate_age],
    chat_model=iModel(model="openai/gpt-4.1-mini"),
)

In [6]:
# Research question
question = "Which company is oldest: Apple, Google, or Microsoft? How much older is the oldest than the newest?"

# Let ReAct figure out the steps
research_result = await researcher.ReAct(
    instruct={"instruction": question}, max_extensions=10, verbose=True
)

print("\n📊 Research Result:")
print(research_result)

Invoking action lookup_company with {'name': 'Google'}.
Action lookup_company invoked, status: completed.
Invoking action lookup_company with {'name': 'Microsoft'}.
Action lookup_company invoked, status: completed.
Invoking action lookup_company with {'name': 'Apple'}.
Action lookup_company invoked, status: completed.


### ReAct Round No.1 Analysis:
```yaml
analysis: To determine which company is the oldest among Apple, Google, and Microsoft, I need to find their respective founding years. With the founding years, I can then identify the oldest and newest among them and calculate how much older the oldest company is compared to the newest one.
planned_actions:
  - action_type: lookup_company
    description: Look up founding years of Apple, Google, and Microsoft to determine which is oldest and calculate age differences.
extension_needed: True
milestone: Identify founding years of Apple, Google, and Microsoft.
action_strategy: concurrent
action_required: True
reason:
  title: Need founding years to compare company ages
  content: The founding years are essential to knowing which company is the oldest and the age difference between the oldest and newest companies.
  confidence_score: 1.0
action_requests:
  - function: lookup_company
    arguments:
      name: Apple
  - function: lookup_company
    arguments:
      name: Google
  - function: lookup_company
    arguments:
      name: Microsoft
action_responses:
  - function: lookup_company
    arguments:
      name: Apple
    output: Apple: Founded 1976, 164000 employees, $394.3B revenue
  - function: lookup_company
    arguments:
      name: Google
    output: Google: Founded 1998, 182000 employees, $307.4B revenue
  - function: lookup_company
    arguments:
      name: Microsoft
    output: Microsoft: Founded 1975, 238000 employees, $211.9B revenue
```
---------

### ReAct Round No.2 Analysis:
```yaml
analysis: The founding years are: Microsoft in 1975, Apple in 1976, and Google in 1998. Microsoft is the oldest company, and Google is the newest. The age difference between Microsoft (oldest) and Google (newest) is 23 years.
planned_actions:
extension_needed: False
milestone: Determine oldest company and age difference from founding years.
action_strategy: concurrent
action_required: False
reason:
  title: Founding years obtained for all companies
  content: Accurate founding years for Apple (1976), Google (1998), and Microsoft (1975) have been found, allowing determination of the oldest company and the age difference correctly.
  confidence_score: 1.0
action_requests:
```
---------

### ReAct Final Answer:
Microsoft is the oldest company, founded in 1975. Google is the newest, founded in 1998. Therefore, Microsoft is 23 years older than Google.
---------


📊 Research Result:
Microsoft is the oldest company, founded in 1975. Google is the newest, founded in 1998. Therefore, Microsoft is 23 years older than Google.


## ReAct vs Regular Function Calling

Let's compare ReAct with regular `operate()` to see the difference:

In [7]:
# Regular function calling approach
regular_branch = Branch(
    system="Answer questions about companies.",
    tools=[lookup_company, calculate_age],
    chat_model=iModel(model="openai/gpt-4.1-mini"),
)

# Same question, different approach
regular_result = await regular_branch.operate(
    instruction=question, actions=True
)

print("🔧 Regular operate() result:")
print(f"\nTools used: {len(regular_result.action_responses)}")

print("\n" + "=" * 50)
print(
    "🧠 ReAct approach showed more reasoning steps and systematic problem solving!"
)

🔧 Regular operate() result:

Tools used: 3

🧠 ReAct approach showed more reasoning steps and systematic problem solving!


## When to Use ReAct

**ReAct is best for:**
- Complex, multi-step problems
- When you want to see the reasoning process
- Problems that need strategic tool use
- Research and analysis tasks

**Regular `operate()` is better for:**
- Simple, single-step actions
- When you just want the answer quickly
- Straightforward tool usage

## Simple ReAct Configuration

Key parameters you can control:

In [8]:
# Different ReAct configurations
demo_branch = Branch(
    system="You solve problems step by step.",
    tools=[add, multiply, divide],
    chat_model=iModel(model="openai/gpt-4.1-mini"),
)

# Quick ReAct (fewer steps)
quick_result = await demo_branch.ReAct(
    instruct={"instruction": "What's 10 * 5 + 3?"},
    max_extensions=10,
    verbose=False,  # Hide reasoning steps
)

print("Quick ReAct:", quick_result)

# Detailed ReAct (more steps)
detailed_result = await demo_branch.ReAct(
    instruct={"instruction": "Calculate (15 * 4) + (20 / 5) - 3"},
    max_extensions=10,
    verbose=True,  # Show reasoning steps
)

print("\nDetailed ReAct:", detailed_result)

Quick ReAct: 10 * 5 + 3 = 50 + 3 = 53
Invoking action multiply with {'a': 15, 'b': 4}.
Action multiply invoked, status: completed.
Invoking action divide with {'a': 20, 'b': 5}.
Action divide invoked, status: completed.


### ReAct Round No.1 Analysis:
```yaml
analysis: The expression is (15 * 4) + (20 / 5) - 3. Based on order of operations, calculate multiplications and divisions first: 15 * 4 and 20 / 5. Then, proceed to additions and subtractions.
planned_actions:
  - action_type: multiply
    description: Calculate 15 * 4 as the first part of the expression.
  - action_type: divide
    description: Calculate 20 / 5 as the second part of the expression.
extension_needed: True
milestone: Complete multiplication and division steps
action_strategy: concurrent
action_required: True
reason:
  title: Stepwise calculation to break down expression
  content: By first calculating the multiplication and division parts, the expression simplifies correctly for the next steps.
  confidence_score: 1.0
action_requests:
  - function: multiply
    arguments:
      a: 15
      b: 4
  - function: divide
    arguments:
      a: 20
      b: 5
action_responses:
  - function: multiply
    arguments:
      a: 15
      b: 4
    output: 60
  - function: divide
    arguments:
      a: 20
      b: 5
    output: 4.0
```
---------

Invoking action add with {'a': 60, 'b': 4.0}.
Action add invoked, status: completed.


### ReAct Round No.2 Analysis:
```yaml
analysis: We have calculated 15 * 4 = 60 and 20 / 5 = 4.0. Now, to follow the expression (15 * 4) + (20 / 5) - 3, we first add 60 and 4.0, then subtract 3.
planned_actions:
  - action_type: add
    description: Add the results of multiplication (60) and division (4.0).
  - action_type: add
    description: Subtract 3 from the sum of previous addition to complete the expression.
extension_needed: True
milestone: Sum multiplication and division results, then subtract 3
action_strategy: sequential
action_required: True
reason:
  title: Stepwise evaluation
  content: By adding the intermediate results first, we simplify the problem and then finish by subtracting 3, ensuring accuracy.
  confidence_score: 1.0
action_requests:
  - function: add
    arguments:
      a: 60
      b: 4.0
action_responses:
  - function: add
    arguments:
      a: 60
      b: 4.0
    output: 64.0
```
---------

Invoking action add with {'a': 64.0, 'b': -3}.
Action add invoked, status: completed.


### ReAct Round No.3 Analysis:
```yaml
analysis: We have the partial sum 64.0 from (15 * 4) + (20 / 5). The last operation is to subtract 3 from 64.0 to finalize the expression (15 * 4) + (20 / 5) - 3.
planned_actions:
  - action_type: add
    description: Subtract 3 from the sum of previous results (64.0) to complete the expression.
extension_needed: True
milestone: Complete subtraction to finish expression evaluation
action_strategy: sequential
action_required: True
reason:
  title: Final subtraction step
  content: Subtracting 3 from the partial sum completes the evaluation of the given expression.
  confidence_score: 1.0
action_requests:
  - function: add
    arguments:
      a: 64.0
      b: -3
action_responses:
  - function: add
    arguments:
      a: 64.0
      b: -3
    output: 61.0
```
---------

### ReAct Round No.4 Analysis:
```yaml
analysis: All calculations are complete: 15 * 4 = 60, 20 / 5 = 4.0, sum is 64.0, and subtracting 3 results in 61.0. We can now provide the final answer.
planned_actions:
  - action_type:

    description: Provide the final answer to the user after completing all calculations.
extension_needed: False
milestone: Provide final answer
action_strategy: sequential
action_required: False
reason:
  title: Final step
  content: The expression has been fully evaluated step-by-step with confidence.
  confidence_score: 1.0
action_requests:
```
---------

### ReAct Final Answer:
(15 * 4) + (20 / 5) - 3 = 60 + 4.0 - 3 = 61.0
---------


Detailed ReAct: (15 * 4) + (20 / 5) - 3 = 60 + 4.0 - 3 = 61.0


## Key Takeaways

**ReAct Pattern:**
1. 🧠 **Think** about what needs to be done
2. 🔧 **Act** using available tools
3. 🔄 **Repeat** until problem is solved

**Benefits:**
- More reliable problem solving
- Transparent reasoning process
- Better handling of complex tasks
- Strategic tool usage

**Next**: You now understand ReAct fundamentals! This prepares you for more advanced reasoning patterns and complex workflows.